In [1]:
import fastf1
import pandas as pd
import os

fastf1.Cache.enable_cache('../data') # Cache in /data

In [2]:
def get_race_laps(year: int, session_name: str = 'British Grand Prix') -> pd.DataFrame:
    """
    Obtiene los datos de vueltas de la carrera de Silverstone para un año dado.
    Devuelve un DataFrame con las columnas: Driver, LapTime, Sector1Time, Sector2Time, Sector3Time.

    Args:
        year (int): Año de la carrera.
        session_name (str): Nombre de la sesión (por defecto 'British Grand Prix').

    Returns:
        pd.DataFrame: DataFrame con los datos de vueltas limpios.
    """
    if not isinstance(year, int) or year < 1950:
        raise ValueError("El año debe ser un entero válido (>=1950).")
    try:
        session = fastf1.get_session(year, session_name, 'R')
        session.load()
    except Exception as e:
        raise RuntimeError(f"Error cargando la sesión: {e}")

    laps = session.laps[["Driver", "LapTime", "Sector1Time", "Sector2Time", "Sector3Time"]].copy()
    # Eliminar filas con valores nulos o tiempos no válidos
    laps = laps.dropna(subset=["Driver", "LapTime", "Sector1Time", "Sector2Time", "Sector3Time"])
    # Opcional: filtrar solo vueltas válidas (LapTime > 0)
    laps = laps[laps["LapTime"].apply(lambda x: hasattr(x, "total_seconds") and x.total_seconds() > 0)]

    return laps.reset_index(drop=True)


In [4]:
def load_session_data(year=2025, session_name='British Grand Prix'):
    session_data = get_race_laps(year, session_name)
    if session_data.empty:
        raise ValueError(f"No se encontraron datos de la carrera para {year} {session_name}.")
    return session_data

session_2025 = load_session_data(2025, 'British Grand Prix')

core           INFO 	Loading data for British Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '27', '44', '1', '10', '18', '23', '14', '63', '87', '55', '31', '16', '22', '12', '6', '5', '30', '43']


In [6]:
session_2025

,Driver,LapTime,Sector1Time,Sector2Time,Sector3Time
0,VER,0 days 00:02:15.598000,0 days 00:00:45.755000,0 days 00:00:53.700000,0 days 00:00:36.143000
1,VER,0 days 00:02:17.357000,0 days 00:00:48.885000,0 days 00:00:50.924000,0 days 00:00:37.548000
2,VER,0 days 00:01:45.568000,0 days 00:00:36.223000,0 days 00:00:40.712000,0 days 00:00:28.633000
3,VER,0 days 00:01:44.809000,0 days 00:00:31.996000,0 days 00:00:43.110000,0 days 00:00:29.703000
4,VER,0 days 00:02:19.789000,0 days 00:00:47.766000,0 days 00:00:56.615000,0 days 00:00:35.408000
...,...,...,...,...,...
711,BEA,0 days 00:01:31.041000,0 days 00:00:29.191000,0 days 00:00:37.115000,0 days 00:00:24.735000
712,BEA,0 days 00:01:31.273000,0 days 00:00:29.030000,0 days 00:00:37.237000,0 days 00:00:25.006000
713,BEA,0 days 00:01:30.921000,0 days 00:00:29.104000,0 days 00:00:37.062000,0 days 00:00:24.755000
714,BEA,0 days 00:01:31.121000,0 days 00:00:29.159000,0 days 00:00:36.990000,0 days 00:00:24.972000


In [8]:
def convert_times_to_seconds(df):
    for col in ["LapTime", "Sector1Time", "Sector2Time", "Sector3Time"]:
        df[f"{col} (s)"] = df[col].dt.total_seconds()
    return df
def average_sector_times_by_driver(df):
    new_df = df.groupby("Driver")[["Sector1Time (s)", "Sector2Time (s)", "Sector3Time (s)"]].mean().reset_index()
    new_df["TotalSectorTime (s)"] = ( new_df["Sector1Time (s)"] + new_df["Sector2Time (s)"] + new_df["Sector3Time (s)"])
    return new_df

In [9]:
session_2025 = convert_times_to_seconds(session_2025)

In [10]:
session_2025

,Driver,LapTime,Sector1Time,Sector2Time,Sector3Time,LapTime (s),Sector1Time (s),Sector2Time (s),Sector3Time (s)
0,VER,0 days 00:02:15.598000,0 days 00:00:45.755000,0 days 00:00:53.700000,0 days 00:00:36.143000,135.598,45.755,53.700,36.143
1,VER,0 days 00:02:17.357000,0 days 00:00:48.885000,0 days 00:00:50.924000,0 days 00:00:37.548000,137.357,48.885,50.924,37.548
2,VER,0 days 00:01:45.568000,0 days 00:00:36.223000,0 days 00:00:40.712000,0 days 00:00:28.633000,105.568,36.223,40.712,28.633
3,VER,0 days 00:01:44.809000,0 days 00:00:31.996000,0 days 00:00:43.110000,0 days 00:00:29.703000,104.809,31.996,43.110,29.703
4,VER,0 days 00:02:19.789000,0 days 00:00:47.766000,0 days 00:00:56.615000,0 days 00:00:35.408000,139.789,47.766,56.615,35.408
...,...,...,...,...,...,...,...,...,...
711,BEA,0 days 00:01:31.041000,0 days 00:00:29.191000,0 days 00:00:37.115000,0 days 00:00:24.735000,91.041,29.191,37.115,24.735
712,BEA,0 days 00:01:31.273000,0 days 00:00:29.030000,0 days 00:00:37.237000,0 days 00:00:25.006000,91.273,29.030,37.237,25.006
713,BEA,0 days 00:01:30.921000,0 days 00:00:29.104000,0 days 00:00:37.062000,0 days 00:00:24.755000,90.921,29.104,37.062,24.755
714,BEA,0 days 00:01:31.121000,0 days 00:00:29.159000,0 days 00:00:36.990000,0 days 00:00:24.972000,91.121,29.159,36.990,24.972


In [16]:
total_times = session_2025.groupby("Driver")["LapTime (s)"].sum().sort_values(ascending=True).reset_index()
total_times

,Driver,LapTime (s)
0,BOR,273.901
1,HAD,1697.303
2,ANT,1929.936
3,PIA,4739.145
4,NOR,4747.068
5,HUL,4788.753
6,VER,4797.165
7,STR,4809.054
8,TSU,4949.714
9,HAM,4950.642


In [17]:
session = fastf1.get_session(2025, 'British Grand Prix', 'R')
session.load()

core           INFO 	Loading data for British Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '27', '44', '1', '10', '18', '23', '14', '63', '87', '55', '31', '16', '22', '12', '6', '5', '30', '43']


In [19]:
session.results

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
4,4,L NORRIS,NOR,norris,McLaren,F47600,mclaren,Lando,Norris,Lando Norris,...,,1.0,1,3.0,NaT,NaT,NaT,0 days 01:37:15.735000,Finished,25.0
81,81,O PIASTRI,PIA,piastri,McLaren,F47600,mclaren,Oscar,Piastri,Oscar Piastri,...,,2.0,2,2.0,NaT,NaT,NaT,0 days 00:00:06.812000,Finished,18.0
27,27,N HULKENBERG,HUL,hulkenberg,Kick Sauber,01C00E,sauber,Nico,Hulkenberg,Nico Hulkenberg,...,,3.0,3,19.0,NaT,NaT,NaT,0 days 00:00:34.742000,Finished,15.0
44,44,L HAMILTON,HAM,hamilton,Ferrari,ED1131,ferrari,Lewis,Hamilton,Lewis Hamilton,...,,4.0,4,5.0,NaT,NaT,NaT,0 days 00:00:39.812000,Finished,12.0
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,4781D7,red_bull,Max,Verstappen,Max Verstappen,...,,5.0,5,1.0,NaT,NaT,NaT,0 days 00:00:56.781000,Finished,10.0
10,10,P GASLY,GAS,gasly,Alpine,00A1E8,alpine,Pierre,Gasly,Pierre Gasly,...,,6.0,6,8.0,NaT,NaT,NaT,0 days 00:00:59.857000,Finished,8.0
18,18,L STROLL,STR,stroll,Aston Martin,229971,aston_martin,Lance,Stroll,Lance Stroll,...,,7.0,7,17.0,NaT,NaT,NaT,0 days 00:01:00.603000,Finished,6.0
23,23,A ALBON,ALB,albon,Williams,1868DB,williams,Alexander,Albon,Alexander Albon,...,,8.0,8,13.0,NaT,NaT,NaT,0 days 00:01:04.135000,Finished,4.0
14,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,,9.0,9,7.0,NaT,NaT,NaT,0 days 00:01:05.858000,Finished,2.0
63,63,G RUSSELL,RUS,russell,Mercedes,00D7B6,mercedes,George,Russell,George Russell,...,,10.0,10,4.0,NaT,NaT,NaT,0 days 00:01:10.674000,Finished,1.0
